# ASTR3110 Computer Laboratory 2: Determining star cluster membership with Gaia data and Gaussian Mixuture Modelling.

In this lab, you will again make use of data from the Gaia Data Release 2 (Gaia DR2). This time you will use the position (R.A., decl.), proper motions (pmRA, pmdec), and distance measurements from Gaia to identify open clusters using Gaussian Mixture Modelling (GMM). You will also identify which stars in the data are most likely gravitationally bound members of the clusters. By identifying which stars in a region of sky belong to different star clusters, it is possible to investigate the properties of the open clusters such as their kinematics, stellar populations etc. 

In the first part of the lab, you will download and then get to know your data, eliminating spurious data points using the techniques you learned in Lab 1, as well as performing task-specific filtering of the data. Then, you will explore the data by producing a series of plots that will help you better understand the data, and will also guide your decision making for future analysis.

In the second part of the lab, you will begin to use the [scikit learn GMM package](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) to identify star clusters in the data. This will require an amount of experimentation, which will help you to become familiar with using the GMM. You will refine your analysis to a point where you can isolate the clusters in the data and use the GMM fits to determine the stars with the greatest likelihood of being gravtitationally bound members of those clusters.

Finally, you will plot separate colour-magnitude diagrams for the clusters you have found in the data.

At the completion of this lab, you will have acquired (or improved) the following skills:
- Data filtering using Pandas.
- Manipulating data to derive required parameters.
- Plotting data and interpreting plots using matplotlib and seaborn.
- Using scikit learn Gaussian Mixture Modelling.

This Lab is to be submitted as a publication-style report (details will appear on iLearn later), which will be worth 25\% of your final grade. You should also submit your notebook alongside your report. In completing your notebook, keep in mind the following pointers for good coding practice, and will also help when it is time to write your report, where you will be expected to explain what you have done during your analysis, and interpret results.
- Comments: Your code should always be commented. You can either do this by adding explanation text placed in blocks just before code blocks OR as comments within the code blocks themselves. Your comments should demonstrate that you understand what your code is doing, and why! 
- Plots should be well presented and explained: E.g., reasonable axes (e.g., ranges should be set so that trends are clearly visible), clear axis labels. You should be able to explain the "why" and "what" of your plots. Why are you plotting this? What does your plot show?
- Formatting of your code (easy to understand, sensible variable names etc.)
- Explanations and/or justification for cuts made on the data.
- Comments/answers to any questions posed in the notebooks as well as comments on interpretation/comparison of results. 

# Part 1: Obtaining the data and applying filters

In this part, we will retrieve the Gaia data from the archive, and weed out obviously bad data.
## Downloading the data

The data for this practical comes from the [Gaia](https://www.cosmos.esa.int/web/gaia/) mission and must be downloaded from the online Gaia archive at [http://gea.esac.esa.int/archive/](http://gea.esac.esa.int/archive/). The tabular data is retrieved from the database via an astronomy data query language (ADQL) statement:

```
SELECT source_id, dist.r_est, dist.r_lo, dist.r_hi, dist.r_len, dist.result_flag, dist.modality_flag, src.ra, src.dec, src.L, src.B, src.parallax, src.parallax_error, src.pmra, src.pmra_error, src.pmdec, src.pmdec_error, src.radial_velocity, src.radial_velocity_error, src.astrometric_chi2_al, src.astrometric_n_good_obs_al, src.phot_g_mean_mag, src.phot_bp_mean_mag,src.phot_rp_mean_mag, src.phot_g_mean_flux, src.phot_bp_mean_flux, src.phot_rp_mean_flux, src.phot_g_mean_flux_error, src.phot_bp_mean_flux_error, src.phot_rp_mean_flux_error, src.phot_bp_rp_excess_factor 
FROM external.gaiadr2_geometric_distance as dist 
JOIN gaiadr2.gaia_source AS src USING (source_id) 
where CONTAINS(POINT('icrs', src.ra, src.dec), CIRCLE('icrs',116.3500,-37.9500,2))=1
```

This query selects multiple columns from the gaia_source and gaiadr2_geometric tables in the archive that are merged using a common ```source_id``` column. The selected data are constrained to a one degree region centred at R.A.=116.35 degrees, decl. = -37.95 degrees. This is centred on the cluster NGC2451, which was discovered in the 1600s and was subject to much debate surrounding whether the stars in this region contained one cluster, or was the superposition of two clusters seen in projected but separated along the line of sight. In this field, there is also the NGC2477 cluster. Your job is to find all of the clusters in this dataset and identify which stars are most likely to be members of these clusters.

Follow the procedure from Lab 1 to obtain the data and read it in as a Pandas data frame.


## Inspecting the data 

In the below tab, explore your data frame using Pandas functions. Again, the meanings of the column names and the units of the data can be found in the [GAIA data model documentation](http://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/) or viewed as hover help text on the sidebar of the GAIA query page. As in Lab 1 (and for all research), an important part of both data science and astro research is *understanding* the data you are using. Using the information at the Gaia webpage, make a new markdown cell where you list each column in the DataFrame, along with a short description of the data contained in the column of interest (be sure to note units in your description). Some of the columns should be familiar from Lab 1, but there are many new columns to get to know.

## Preliminary Filtering of the Data

Now that we know what data is contained within the dataframe, we can remove those rows with missing/Null/NaN/Inf values as well as any unnecessary columns as per Lab 1.


## Task-specific filtering of the data

As outlined above, we will be using the Gaia data to find clusters using positions, proper motions, and parallax measurements (collectively known as astrometry). This means that we require that the data have reliable, high signal-to-noise values so that spurious results do not affect the clustering outcomes. We will also be using the photometric data to plot colour-magnitude diagrams for the detected clusters. 

The selection of high-quality astrometric measurements is discussed in detail in Appendix C of [Lindegren et al. (2018)](https://ui.adsabs.harvard.edu/abs/2018A%26A...616A...2L/abstract). In particular, equation C.1 in Lindegren et al. (2018) is:
$$
\mu < 1.2\times {\rm max}(1,exp(-0.2(G -19.5))
$$
where $G$ is the Gaia $G$-band magnitude, $\mu = \sqrt{\chi^2/\nu}$, $\nu = {\rm N_{good}} - 5$ is the number of degrees of freedom of the fit used to derive the position, parallax, and proper motion from the observations. This selection helps to weed out stars for which the astrometric solution used to determine position, parallax, and proper motion has failed due to, e.g., unresolved companion stars that affect the measuremnts.

Furthermore, unresolved companion stars can impact the measured photometry. Lindegren et al. (2018) uses the ``flux excess factor'' (E=phot_bp_rp_excess_factor), which compares the $BP$-, $RP$-, and $G$-band fluxes in order to determine the criterion for removing sources with adversely affected photometry. It is given by their equation C.2:
$$
1.0+0.015(G_{BP}-G_{RP})^2 < E < 1.3+0.06(G_{BP} - G_{RP})^2
$$
This works because the $G$ flux is measured through a smaller region than the $BP$ and $RP$ fluxes, so any excess flux in the latter two measurements means there is likely an unresolved star or some other problem with the measurements (see [Evans et al. 2018](https://ui.adsabs.harvard.edu/abs/2018A%26A...616A...4E/abstract) for more information). These problematic sources need to be removed.

Use the above two criteria to filter your data. You should also filter your data so that the signal-to-noise (as determined by the $X/\sigma_X$ where $X$ is the measurement of interest and $\sigma_{X}$ its error) of the parallax, BP flux, and RP flux measurements is greater than 10.

# Exploration of the data

Begin exploring your filtered data to see if you can spot any clusters "by eye". Start off by exploring the data using scatter plots, which you should be familiar with from Lab 1. Keep in mind that the stars that are in clusters are expected to be found in a relatively small volumes of space, thus, stars in clusters will have very similar distances (with some spread due to distance uncertainties), and may appear as local enchancements in the projected number density of stars on the sky (R.A. and decl.; although this is not always the case!). 

Importantly, the stars in the cluster all move together, thus, stars bound to open and globular clusters will form compact overdensities in proper motion space. On the other hand, "field" stars that are not bound to clusters will be more evenly spread throughout proper motion space.

It is likely that you will soon find that scatter plots are often saturated and, therefore, are inedequate without a sensible partition of the data (e.g., you could plot different ranges in distance separately). You may wish to use 2D histograms to represent the data in some of your plots, which can be accomplished with the [hist2D](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist2d.html) capability of matplotlib pyplot. You should also explore using the [Seaborne](https://seaborn.pydata.org/index.html) plotting package, which hosts a number of powerful plotting tools for visualising 2D data (e.g., [jointplots](https://seaborn.pydata.org/generated/seaborn.jointplot.html#seaborn.jointplot), [kdeplot](https://seaborn.pydata.org/generated/seaborn.kdeplot.html#seaborn.kdeplot) and [others](https://seaborn.pydata.org/examples/index.html)). 

There is one cluster that should become obvious in the plots showing R.A. vs decl. ([NGC2477](https://en.wikipedia.org/wiki/NGC_2477)). Manually isolate the stars in this cluster by selecting stars in the approprate R.A. and decl. range, and explore its properties by plotting the stars separately onto the entire distribution (e.g., plots showing the the distances, the proper motions etc.)

# <font color=red>You should aim to get to here by the end of the first week of this lab!</font>

# Part 2: Running GMM

In this part, you will begin using Gaussian Mixture Modelling to try to detect clusters, and to identify which stars in the dataset are most likely to belong to those clusters.


## Initial Experimentation with GMM

In the above exploratory phase, the NGC2477 cluster was visible as a localised region with a high number density of stars. In the Week 7 Lectorial, we learned about how the [scikit learn Gaussian Mixture Models](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) (see also [here](https://scikit-learn.org/stable/modules/mixture.html#gmm))  can be used to partition data into a number of different components, and then predict which data points belong to the different components. Using your analysis above to guide you, create a dataframe that contains stars in the *distance range* that encompasses the NGC2477 cluster (only select based on distance (r_est) at this stage). This dataset can be used to get a handle on how the GMM process works. Perform the following tasks on your new dataset.

### Step 1: 
Start by running GMM on only the R.A. and decl., using 2 components (that is, use two gaussian components to fit the data). Explore the outputs: examine the "means_", "covariances_", and use the "predict" method to assign membership of stars to the 2 components. Plot the results as a scatterplot, using different colours to highlight stars belonging to the different components. Do the results make sense? Do you need more components?

### Step 2: 
Repeat Step 1, but now include R.A., decl., pmRA , and pmdec in the fit. How does the result compare to that in step 1? 

### Step 3: 
Here, we want to determine how many components are required to fit the data without "overfitting". Repeat step 2, but code up a loop that increases the number of components fitted by GMM. Determine the "Bayesian Information Criteria", or [BIC](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture.bic) (discussed in [Week 3 lectorial](https://github.com/MQ-ASTR3110-2021/ASTR3110_Tutorial_Notebooks_2021/blob/master/Solution_Notebooks/ASTR3110_Tutorial_3_Fitting.ipynb) and [here](https://en.wikipedia.org/wiki/Bayesian_information_criterion)) for each different fit. Report the results by printing to screen. We can use the BIC to determine the optimal number of components that are required to improve the fit without overfitting. This is done by ensuring that in going from a smaller to larger number of components, the BIC decreases by at least 6. Save your BIC values, and plot the the BIC value as a function of the number of components. Also plot the *change* in BIC as the number of components is increased. Use this to determine the optimum number of components, i.e, find the component number where if you incerease that number by one, the change in the BIC no longer *decreases* by more than 6.

### Step 4: 
Repeat the GMM fit using the optimum number of components determined in Step 3 -- explore the outputs as was done in Step 1. Note how the properties of the identified components are different. In particular, the NGC2447 cluster should have a substantially smaller dispersion (< 1mas/yr) in the pmRA and pmdec -- this is the distinguishing characteristic that allows us to detect clusters in these data. The dispersion can be determined by examining the square root of the diagonal terms in the covariance matrix for each component.

## Finding the other clusters <font color=red>(you should aim to have made a good start at this part by the end of week 2!)</font>

Now that you have used GMM to identify NGC2477, as well as determine which stars are most likely members, you can find the other two clusters in the data (NGC2451A and NGC2451B). These two clusters are not at all obvious as overdensities in R.A. and decl. because they are lost in the background of field stars. We can find these clusters in a semi-automated fashion by running GMM as outlined above, but now in separate distance bins, and looking for the compact clusterings in proper motion that we identified for NGC2477. The segregation by distance helps to increase the contrast of the cluster stars against the backdrop of the field stars, which in turn allows GMM to more easily locate the star clusters in RA, dec, pmRA and pmdec.

To do this, you will need design an algorithm that does the following:
- Generate bins in distance, e.g., 8 distance bins with width 250pc so that the first bin runs from 0-250pc, the second 250-500pc, and so forth.
- For each distance bin select the subset of stars within that distance range. 
- On the subset of stars, run your GMM algorithm on RA, dec, pmra, pmdec, using the BIC to determine the optimal number of components required.
- Generate appropriate plots and print any results/outputs that will help you to understand which distance ranges contain clusters, and what their properties are.


## Finalising the clusters and determining membership.

Use the results from the previous section to select appropriate distance ranges for the 3 clusters. Select the subset of stars within those distance ranges and rerun GMM on each subset in order to refine the membership. You should now only need to fit for 2 components; one for the cluster stars and one for the field stars. For this final part, you should include the distance (r_est) in the GMM fit in order to get the best membership constraint for the stars. 

You should now be able to assign the stars in this region to either the field or one of the NGC2451A, NGC2451B or NGC2477 clusters. Use your final membership to plot the distribution of clusters stars in position (R.A., dec), proper motion (pmRA, pmdec), and distance in comparision to non-cluster stars.

You can also use your final samples of member stars to determine important parameters for each of the clusters such as mean distance, total number of stars, mean position etc. These parameters can be recorded in your writeup, along with any other parameters you might think are important.

# Part 3: Cluster colour magnitude diagrams


With knowledge of which stars are most likely to be members of the three star clusters in this field, you can now plot the colour-magnitude diagrams (CMD; $G_{BP} - G_{RP}$ vs $M_G$)  separately for the cluster and non-cluster stars. When determining the absolute G-band magnitude, $M_G$, for the cluster stars you can use the mean distance to the cluster as determined above.

In your write-up of this prac, you should compare the CMDs of the three clusters, explaining any differences seen. You should also compare the cluster CMDs to that of the field sample of stars, again explaining differences that you see. 